<a href="https://colab.research.google.com/github/ayushjain1144/semantic-segmentation-IGCAR/blob/master/bert_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/google-research/bert

Cloning into 'bert'...
remote: Enumerating objects: 329, done.
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 239.27 KiB | 7.98 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/igcar_ps/ /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [18]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
#import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")



from bert import modeling, optimization, tokenization
#from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()

log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s : \
    %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    
    with tf.Session(TPU_ADDRESS) as session:
        log.info('TPU address is ' + TPU_ADDRESS)
        with open('/content/adc.json', 'r') as f:
          auth_info = json.load(f)
        
        tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
        
       
else:
    log.warning('Not connected to TPU runtime')
    USE_TPU = False

2019-07-01 10:19:57,695 :     Not connected to TPU runtime
W0701 10:19:57.695699 139813024868224 interactiveshell.py:2882] Not connected to TPU runtime


In [5]:
BUCKET_PATH = "gs://ayushjain1144-bucket/"
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = BUCKET_PATH + BERT_MODEL
print('****** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR)) 
#!ls BERT_PRETRAINED_DIR

****** BERT pretrained directory: gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12 *****


In [0]:
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt.*'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'

In [0]:
QA_DATASET = BUCKET_PATH + 'qa_dataset'
QA_TRAIN_FILE = QA_DATASET + '/train-v1.1.json'
QA_PREDICT_FILE = QA_DATASET + '/dev-v1.1.json'


In [0]:
OUTPUT_DIR = BUCKET_PATH + 'output-dir/'

In [0]:
TRAIN_BATCH_SIZE = 16
LEARNING_RATE = 3e-5
NUM_TRAIN_EPOCHS = 2.0
MAX_SEQ_LENGTH = 256
DOC_STRIDE = 128
SAVE_CHECKPOINT_STEPS=1000
ITERATIONS_PER_LOOP=1000


In [0]:
from bert.run_squad import input_fn_builder, model_fn_builder

In [0]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8

In [20]:
from bert.run_squad import read_squad_examples

train_examples = read_squad_examples(input_file=QA_TRAIN_FILE,is_training=True)

print(len(train_examples))

UnrecognizedFlagError: ignored

In [0]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

model_fn_builder(bert_config=bert_config,
                init_checkpoint=INIT_CHECKPOINT,
                learning_rate=LEARNING_RATE,
                num_)

In [21]:
XARGS_CMD = ("python3 bert/run_squad.py "
            "--vocab_file={} "
            "--bert_config_file={} "
            "--init_checkpoint={} "
            "--do_train=True "
            "--train_file={} "
            "--do_predict=True "
            "--predict_file={} "
            "--train_batch_size=12 "
            "--learning_rate=3e-5 "
            "--num_train_epochs=2.0 "
            "--max_seq_length=256 "
            "--doc_stride=128 "
            "--output_dir={} "
            )
XARGS_CMD = XARGS_CMD.format(VOCAB_FILE, 
                            BERT_CONFIG,
                            INIT_CHECKPOINT,
                            QA_TRAIN_FILE,
                            QA_PREDICT_FILE,
                            OUTPUT_DIR)
                           
print(XARGS_CMD)

python3 bert/run_squad.py --vocab_file=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/bert_model.ckpt --do_train=True --train_file=gs://ayushjain1144-bucket/qa_dataset/train-v1.1.json --do_predict=True --predict_file=gs://ayushjain1144-bucket/qa_dataset/dev-v1.1.json --train_batch_size=12 --learning_rate=3e-5 --num_train_epochs=2.0 --max_seq_length=256 --doc_stride=128 --output_dir=gs://ayushjain1144-bucket/output-dir/ 


In [0]:
!$XARGS_CMD

W0701 10:36:31.652795 139853665920896 deprecation_wrapper.py:119] From /content/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0701 10:36:31.655218 139853665920896 deprecation_wrapper.py:119] From bert/run_squad.py:1283: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0701 10:36:31.655867 139853665920896 deprecation_wrapper.py:119] From bert/run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0701 10:36:31.656021 139853665920896 deprecation_wrapper.py:119] From bert/run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0701 10:36:31.656171 139853665920896 deprecation_wrapper.py:119] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0701 10:36:32.816871 139853665920896 deprecation_wrapper.py:119] Fro